# <span style='color:DarkMagenta'> Lesson 1 Exercise 2: Creating a Table with Apache Cassandra </span>


### Walk through the basics of creating a table in Apache Cassandra
- inserting rows of data
- doing simple CQL query to validate the information

### <span style='color:Crimson'> Note: I wasn't able to install Cassandra locally so that's why there are errors when running the cells! </span>


In [2]:
# use Python module for working with Cassandra database --> cassandra-driver
# this you would first install locally

# pip install cassandra-driver

# import apache cassandra python package
import cassandra

In [11]:
# First, create a connection to the database
# this connects to our local instance of apache cassandra 
# this connection will reach out to the database and insure we have the correct privilages to connect to this database

In [12]:
# Interaction with Cassandra database is done through Cluster object

from cassandra.cluster import Cluster # Cassandra.cluster module defines Cluster class

try:
    # We first need to declare Cluster object
    cluster = Cluster(['127.0.0.1']) # to connect to locally installed apache cassandra instance
    session = cluster.connect() # then we need to connect and that will create the session to execute queries
    
except Exception as e:
    print(e)
    
# get an error because I was unable to install cassandra in windows 10
# but if it was installed correctly, then it would run and connect to the local instance

('Unable to connect to any servers', {'127.0.0.1:9042': ConnectionRefusedError(10061, "Tried connecting to [('127.0.0.1', 9042)]. Last error: No connection could be made because the target machine actively refused it")})


In [13]:
# to test the connection run a simple query
# note: do not do this in actual nosql tables - would be too many rows to query

test_query = "select * from music_library"

try:
    session.execute(test_query)
except Exception as e:
    print(e)


name 'session' is not defined


In [14]:
# Next, need to create a keyspace
# keyspace is like the database made up of one or more tables

# for now, ignore the replication strategy and factor information 
# will go into more detail in later session

# on a one node local instance this will be the strategy and replication factor 
# replication_factor == how many times your data is copied 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
        {'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
    ) 
    
except Exception as e:
    print(e)



name 'session' is not defined


In [15]:
# once keyspace is created, you can connect to it

In [16]:
# connect to keyspace
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)
    

name 'session' is not defined


In [ ]:
# now that the keyspace is created and you're connected
# we can create tables

### SCENARIO:

We would like to start creating a music library of albums. Each album has album name, artist name, and year

**BUT**

Apache cassandra is a nosql database. This means we can't model our data and create our table without more information.

**THE FIRST THING TO ASK IS:**

*what queries will be performed on this data?*

**In this case, I woud like to be able to get every album that was released in a particular year**

`select * from music_library where year = 1970`

Because of this, we need to be able to do a `WHERE` on `YEAR`
- `YEAR` will become my partition key, and `artist name` will be the clustering column to make each primary key unique

***Remember: there are no duplicates in cassandra - assume an artist doesn't make more than one album a year***

### So then:
- table name: music_library
- column 1: album name
- column 2: artist name
- column 3: year
- PRIMARY KEY(year, artist name) --> partition by year, and include artist name to make it unique

### Now to translate this information into a create table statement


In [10]:
# create table
create_query = "CREATE TABLE IF NOT EXISTS music_library "
create_query = create_query + "(year int, artist_name text, album_name text, PRIMARY KEY (year, artist_name))"

print(create_query)

# run this query
try:
    session.execute(create_query)
except Exception as e:
    print(e)


CREATE TABLE IF NOT EXISTS music_library (year int, artist_name int, album_name text, PRIMARY KEY (year, artist_name))
name 'session' is not defined


In [ ]:
# check that the table was created
# run a count query (should return 0 because there are no rows in the table)
# *** only do this when we know the table would be empty, otherwise this is not a query to run in nosql tables 
# in some versions of cassandra, it might not return 0 and instead return an error 

count_query = "select count(*) from music_library"
count = session.execute(count_query)

print(count.one())

In [ ]:
# now we can insert rows

In [ ]:
# insert query
insert_query = "INSERT INTO music_library (year, artist_name, album_name)"
insert_query = insert_query + " VALUES (%s, %s, %s)"

try:
    session.execute(insert_query, (1970, "The Beatles", "Let it Be"))
except Exception as e:
    print(e)
    
try:
    session.execute(insert_query, (1965, "The Beatles", "Rubber Soul"))
except Exception as e:
    print(e)
    
# now we should have 2 rows in the table
# because apache cassandra does not allow duplicates, there will only be 2 records no matter how many times we rerun the queries above

In [17]:
# validate that data was inserted

# you can run select * from music_library 
# *** only run this query because we know the table only has 2 rows
# do not run for actual tables with a lot of data

select_query = "select * from music_library"

rows = session.execute(select_query)

for row in rows:
    print (row.year, row.album_name, row.artist_name)

# if you were running the query in the cqlsh (shell program to run cql queries), for loop would NOT be required to print


NameError: name 'session' is not defined

### Validate the data model with our original query

### `select * from music_library where year = 1970`

In [ ]:
test_query = "select * from music_library where year = 1970"

rows = session.execute(test_query)
for row in rows:
    print(row.year, row.album_name, row.artist_name)


In [ ]:
# to end you can drop the table

# to drop the table
drop_query = "drop table music_library"
session.execute(drop_query)

In [ ]:
# finally, close the session and cluster connection

In [ ]:
session.shutdown()
cluster.shutdown()